# Project Notebook: The Linear Regression Model

## 1. Introduction

We started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features. In this project, you can practice what you learned by exploring ways to improve the models we built.

You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

**Tasks**

1. Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well.
2. Read `AmesHousing.tsv` () into a pandas data frame.
3. For the following functions, we recommend creating them in the first few cells in the notebook. This way, you can add cells to the end of the notebook to do experiments and update the functions in these cells.
* Create a function named `transform_features()` that, for now, just returns the train data frame.
* Create a function named `select_features()` that, for now, just returns the Gr Liv Area and SalePrice columns from the train data frame.
* Create a function named `train_and_test()` that, for now:

1. Selects the first 1460 rows from from data and assign to train.
2. Selects the remaining rows from data and assign to test.
3. Trains a model using all numerical columns except the SalePrice column (the target column) from the data frame returned from `select_features()`
4. Tests the model on the test set and returns the `RMSE` value.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 1000
data = pd.read_csv('/content/AmesHousing.tsv', delimiter='\t')


In [ ]:
def transform_features(df):
    return df

# create initial function to select features
def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

# create function to train and test the model
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    #select only the numerical columns
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    # drop target column from training
    features = numeric_train.columns.drop('SalePrice')
    features = numeric_train.columns.drop("SalePrice")

    
    # train and test the model
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])

    pred = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], pred)
    rmse = mse**0.5
    return rmse

# test the functions and check the final output
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

## 2. Feature Engineering

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update `transform_features()` so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. You also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

* remove features that we don't want to use in the model, just based on the number of missing values or data leakage.
* transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc).
* create new features by combining other features.

Next, you need to get more familiar with the remaining columns by reading the data documentation for each column, determining what transformations are necessary (if any), and more. As we mentioned earlier, succeeding in predictive modeling (and competitions like Kaggle) is highly dependent on the quality of features the model has. Libraries like scikit-learn have made it quick and easy to simply try and tweak many different models, but cleaning, selecting, and transforming features are still more of an art that requires a bit of human ingenuity.

**Tasks**

1. As we mentioned earlier, we recommend adding some cells to explore and experiment with different features (before rewriting these functions).

2. The `transform_features()` function shouldn't modify the train data frame and instead return a new one entirely. This way, we can keep using train in the experimentation cells.

3. Which columns contain less than 5% missing values?
* For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.

4. What new features can we create, that better capture the information in some of the features?
* An example of this would be the `years_until_remod` feature we created in the last lesson.

5. Which columns need to be dropped for other reasons?
* Which columns aren't useful for machine learning?
* Which columns leak data about the final sale?

In [ ]:

# drop column with missing values
missing = data.isnull().sum()
missing_col = (missing[missing > len(data)/20]).index
data = data.drop(missing_col, axis=1)

In [ ]:
# drop text columns with 1 or more missing values
text_col = data.select_dtypes(include=['object']).columns
missing_text = data[text_col].isnull().sum()
ms_text_col = missing_text[missing_text >= 1].index
data = data.drop(ms_text_col, axis=1)

In [ ]:
#fill missing value in numerical column with the most frequent value
num_col = data.select_dtypes(include=['integer','float']).columns
num_miss = data[num_col].isnull().sum()
num_miss_cols = num_miss[num_miss > 0].index
data[num_miss_cols] = data[num_miss_cols].fillna(data[num_miss_cols].mode().iloc[0])
data.isnull().sum().sort_values()

Order              0
Central Air        0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
                  ..
Foundation         0
BsmtFin SF 1       0
BsmtFin SF 2       0
Sale Condition     0
SalePrice          0
Length: 64, dtype: int64

In [ ]:
data['years_sold'] = data['Yr Sold'] - data['Year Built']

# check for incorrect value
data['years_sold'][data['years_sold'] < 0] 

2180   -1
Name: years_sold, dtype: int64

In [ ]:
# create new feature to indicate the amount of years until it's being renovated from sale time
data['years_since_remod'] = data['Yr Sold'] - data['Year Remod/Add']

# check for incorrect value
data['years_since_remod'][data['years_since_remod'] < 0]

1702   -1
2180   -2
2181   -1
Name: years_since_remod, dtype: int64

In [ ]:
# drop the incorrect values from previous step
data = data.drop([2180, 1702, 2180, 2181], axis=0)

# remove the original column (not needed anymore)
data = data.drop(['Yr Sold', 'Year Remod/Add', 'Year Built'], axis=1)

In [ ]:
# drop the columns that are not useful for the model
data = data.drop(['PID', 'Order'], axis=1)

# drop the columns that leak information about the sale
data = data.drop(['Mo Sold', 'Sale Type', 'Sale Condition'], axis=1)

In [ ]:
def transform_features(df):
    # drop column with 5% or more missing values
    missing = df.isnull().sum()
    missing_col = (missing[missing > len(df)/20]).index
    df = df.drop(missing_col, axis=1)
    
    # drop text columns with 1 or more missing values
    text_col = df.select_dtypes(include=['object']).columns
    missing_text = df[text_col].isnull().sum()
    ms_text_col = missing_text[missing_text >= 1].index
    df = df.drop(ms_text_col, axis=1)
    
    #fill missing value in numerical column with the most frequent value
    num_col = df.select_dtypes(include=['integer','float']).columns
    num_miss = df[num_col].isnull().sum()
    num_miss_cols = num_miss[num_miss > 0].index
    df[num_miss_cols] = df[num_miss_cols].fillna(df[num_miss_cols].mode().iloc[0])
    
    # create new features
    df['years_sold'] = df['Yr Sold'] - df['Year Built']
    df['years_since_remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    df = df.drop([2180, 1702, 2180, 2181], axis=0)

    # drop not needed & leaking columns
    df = df.drop(['Yr Sold', 'Year Remod/Add', 'Year Built', 'PID', 'Order', 'Mo Sold',
                  'Sale Type', 'Sale Condition'], axis=1)

    return df

# test the function
df = pd.read_csv('/content/AmesHousing.tsv', delimiter='\t')
transformed = transform_features(df)
selected_features = select_features(transformed)
test = train_and_test(selected_features)
test

55275.36731241307

## 3. Feature Selection

Now that we have cleaned and transformed a lot of the features in the data set, it's time to move on to feature selection for numerical features.

**Tasks**

1. Generate a correlation heatmap matrix of the numerical features in the training data set.
* Which features correlate strongly with our target column, `SalePrice`?
* Calculate the correlation coefficients for the columns that seem to correlate well with `SalePrice`. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.

2. Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:
* If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
* Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

3. Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

4. What are some ways we can explore which categorical columns "correlate" well with `SalePrice`?

5. Update the logic for the `select_features()` function. This function should take in the new, modified train and test data frames that were returned from `transform_features()`.

In [ ]:
# check the numerical columns
num_df = transformed.select_dtypes(include = ['float', 'integer'])
num_df


,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_sold,years_since_remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,2.0,528.0,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1.0,730.0,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1.0,312.0,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,2.0,522.0,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,2.0,482.0,212,34,0,0,0,0,0,189900,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80,7937,6,6,0.0,819.0,0.0,184.0,1003.0,1003,0,0,1003,1.0,0.0,1,0,3,1,6,0,2.0,588.0,120,0,0,0,0,0,0,142500,22,22
2926,20,8885,5,5,0.0,301.0,324.0,239.0,864.0,902,0,0,902,1.0,0.0,1,0,2,1,5,0,2.0,484.0,164,0,0,0,0,0,0,131000,23,23
2927,85,10441,5,5,0.0,337.0,0.0,575.0,912.0,970,0,0,970,0.0,1.0,1,0,3,1,6,0,0.0,0.0,80,32,0,0,0,0,700,132000,14,14
2928,20,10010,5,5,0.0,1071.0,123.0,195.0,1389.0,1389,0,0,1389,1.0,0.0,1,0,2,1,6,1,2.0,418.0,240,38,0,0,0,0,0,170000,32,31


In [ ]:
corr = num_df.corr()['SalePrice'].abs().sort_values()
corr

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
years_since_remod    0.534985
Full Bath            0.546118
years_sold           0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [ ]:
# filter the columns with the correlation > 0.4
corr = corr[corr > 0.4] 
corr

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
years_since_remod    0.534985
Full Bath            0.546118
years_sold           0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [ ]:
# drop the low correlating columns
transformed = transformed.drop(corr[corr < 0.4].index, axis=1)

In [ ]:
# Create a list of column names from documentation that should be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

# list the columns that need to be transformed
trans_col = []
for i in nominal_features:
    if i in transformed.columns:
        trans_col.append(i)

# find out the unique values in each column
unique_counts = transformed[trans_col].apply(lambda x: len(x.value_counts())).sort_values()

# set the threshold for the amount of unique values. Here we will use column with <10 unique values.
nonunique_counts = unique_counts[unique_counts > 10]

# drop the columns with unique values >10
transformed = transformed.drop(nonunique_counts.index, axis=1) 

In [ ]:
# select the remaining text columns and convert it to categorical data
text_cols = transformed.select_dtypes(include=['object'])

for i in text_cols:
    transformed[i] = transformed[i].astype('category')

# create dummy columns and drop the original columns
transformed = pd.concat([transformed,
                         pd.get_dummies(transformed.select_dtypes(include=['category']))
                        ], axis =1).drop(text_cols, axis=1)

In [ ]:
def select_features(df, corrval=0.4, threshval=10):
    # check the numerical columns
    num_df = df.select_dtypes(include = ['float', 'int'])
    
    # build the correlation with target column
    corr = num_df.corr()['SalePrice'].abs().sort_values()
    
    # drop the low correlating columns
    df = df.drop(corr[corr < corrval].index, axis=1)
    
    # List the categorical columns
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    # list the columns that need to be transformed
    trans_col = []
    for i in nominal_features:
        if i in df.columns:
            trans_col.append(i)
            
    # find out the unique values in each column
    unique_counts = df[trans_col].apply(lambda x: len(x.value_counts())).sort_values()

    # set the threshold for the amount of unique values
    nonunique_counts = unique_counts[unique_counts > threshval]

    # drop the columns with unique values exceeding threshold value
    df = df.drop(nonunique_counts.index, axis=1) 
    
    # select the remaining text columns and convert it to categorical data
    text_cols = df.select_dtypes(include=['object'])

    for i in text_cols:
        df[i] = df[i].astype('category')

    # create dummy columns and drop the original columns
    df = pd.concat([df,
                         pd.get_dummies(df.select_dtypes(include=['category']))
                        ], axis =1).drop(text_cols, axis=1)

    return df

# test the function
df = pd.read_csv('/content/AmesHousing.tsv', delimiter='\t')
transformed = transform_features(df)
selected_features = select_features(transformed)
test = train_and_test(selected_features)
test

33367.28718340236

## 4. Train and Test

Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's add a parameter named `k` that controls the type of cross validation that occurs.

**Tasks**

1. The optional `k` parameter should accept integer values, with a default value of `0`.

2. When `k` equals `0`, perform holdout validation (what we already implemented):

* Select the first `1460` rows and assign to `train`.
* Select the remaining rows and assign to test.
* Train on `train` and `test` on test.
* Compute the `RMSE` and return.

3. When k equals 1, perform simple cross validation:

* Shuffle the ordering of the rows in the data frame.
* Select the first 1460 rows and assign to `fold_one`.
* Select the remaining rows and assign to `fold_two`.
* Train on `fold_one` and test on `fold_two`.
* Train on `fold_two` and test on `fold_one`.
* Compute the average RMSE and return.

4. When `k` is greater than `0`, implement k-fold cross validation using `k` folds:

* Perform `k-fold` cross validation using k folds.
* Calculate the average `RMSE` value and return this value.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold


# The function accepts k parameter, k=0 (default) for holdout validation, k=1 for cross validation,
# and k fold validation
def train_and_test(df, k=0):
    num_df = df.select_dtypes(include=['float', 'int'])
    features = df.columns.drop('SalePrice')
    lr = LinearRegression()

    if k==0:
        train = df[:1460]
        test = df[1460:]
    
        # train and test the model
        lr.fit(train[features], train["SalePrice"])
        pred = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], pred)
        rmse = np.sqrt(mse)
        
    elif k==1:
        # randomize order of rows
        np.random.seed(1)
        shuffled_index = np.random.permutation(df.index)
        df = df.reindex(shuffled_index)
        
        train = df[:1460]
        test = df[1460:]
    
        # train and test the model
        lr.fit(train[features], train["SalePrice"])
        pred1 = lr.predict(test[features])
        mse1 = mean_squared_error(test['SalePrice'], pred1)
        rmse1 = np.sqrt(mse1)
        
        lr.fit(test[features], test["SalePrice"])
        pred2 = lr.predict(train[features])
        mse2 = mean_squared_error(train['SalePrice'], pred2)
        rmse2 = np.sqrt(mse2)
        
        rmse = (rmse1 + rmse2) / 2
        
    else:
        kf = KFold(n_splits=k, shuffle=True, random_state=1)
        mses = cross_val_score(estimator=lr, X=df[features], y=df['SalePrice'], scoring='neg_mean_squared_error', cv=kf)
        rmse = np.mean(abs(mses)**0.5)

    return rmse

df = pd.read_csv('/content/AmesHousing.tsv', delimiter='\t')
transformed = transform_features(df)
selected_features = select_features(transformed)
test = train_and_test(selected_features, k=5)
test

28832.5142496494

## 5. Next Steps

That's it for the guided steps. Here's some potenial next steps that you can take:

1. Continue iteration on feature engineering:
* Research some other approaches to feature engineering online around housing data.
* Visit the Kaggle kernels [page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels) page for this dataset to see approaches others took.

2. Improve your feature selection:
* Research ways of doing feature selection better with categorical columns (something we didn't cover in this particular course).